# Data Aggregation and Compression Pipeline for Trial-Based Analysis

## Σύνοψη Διαδικασίας

Αυτό το notebook υλοποιεί μια διαδικασία συγκέντρωσης και συμπίεσης πειραματικών δεδομένων που προέρχονται από πολλαπλές παρατηρήσεις ανά πειραματικό trial. Τα αρχικά δεδομένα είναι σε μορφή "long format" όπου κάθε γραμμή αντιπροσωπεύει μια μοναδική παρατήρηση, και ο στόχος είναι η μετατροπή τους σε "wide format" όπου κάθε γραμμή αντιστοιχεί σε ένα μοναδικό **trial_id**. Αυτή η αναδιοργάνωση είναι απαραίτητη για την επεξεργασία που ακολουθεί στο data cleaning pipeline και τελικά για τη δημιουργία του μοντέλου μηχανικής μάθησης.

## Φόρτωση και Αρχική Διερεύνηση

Η διαδικασία ξεκινά με τη φόρτωση του αρχείου **`model_dataset.csv`**, το οποίο περιέχει τα raw δεδομένα όπως εξήχθησαν απευθείας από τη βάση δεδομένων (πιθανότατα Neo4j, βάσει των imported βιβλιοθηκών). Το αρχείο αυτό είναι σε μη συμπιεσμένη μορφή, πράγμα που σημαίνει ότι για κάθε πειραματικό trial μπορεί να υπάρχουν πολλαπλές γραμμές, ανάλογα με το πόσα γονίδια, pathways ή άλλες σχέσεις συνδέονται με αυτό το trial.

Το πρώτο βήμα είναι η διερεύνηση του μεγέθους του dataset. Συγκεκριμένα, καταγράφεται το συνολικό πλήθος γραμμών και το πλήθος των μοναδικών `trial_id` που εμφανίζονται. Αυτή η σύγκριση είναι κρίσιμη για να κατανοήσουμε το επίπεδο της πολυπλοκότητας: αν για παράδειγμα έχουμε 10,000 γραμμές αλλά μόνο 500 μοναδικά trials, σημαίνει ότι κατά μέσο όρο υπάρχουν 20 παρατηρήσεις ανά trial. Αυτές οι πολλαπλές παρατηρήσεις προέρχονται συνήθως από:
- Πολλαπλά γονίδια που συνδέονται με το κάθε trial
- Πολλαπλά biological pathways που εμπλέκονται
- Πολλαπλές TPM (Transcripts Per Million) μετρήσεις για διαφορετικά γονίδια

Επιπλέον, πραγματοποιείται έλεγχος για null τιμές στο αρχικό dataset, ώστε να διαπιστωθεί αν υπάρχουν ελλείψεις δεδομένων ήδη από τη φάση εξαγωγής. Ο έλεγχος αυτός γίνεται τόσο σε επίπεδο συνολικής ύπαρξης NaN values (`isnull().values.any()`), όσο και σε επίπεδο ανά στήλη (`isnull().sum()`), για να εντοπιστεί ποιες μεταβλητές έχουν missing data.

## Στρατηγική Συμπίεσης με Aggregation

Το κεντρικό βήμα της διαδικασίας είναι η **aggregation των δεδομένων με βάση το trial_id**. Αυτή η τεχνική, που υλοποιείται με τη μέθοδο `groupby()` του pandas, συγκεντρώνει όλες τις γραμμές που ανήκουν στο ίδιο trial και τις συνδυάζει σε μία μόνο γραμμή.

Η προσέγγιση που υιοθετείται είναι η μετατροπή όλων των τιμών κάθε στήλης σε **λίστες**. Συγκεκριμένα, για κάθε στήλη εκτός από το `trial_id`, δημιουργείται ένα aggregation dictionary που ορίζει ότι όλες οι τιμές θα συγκεντρωθούν σε μια λίστα (`agg_dict={col:list for col in cols_to_agg}`). Για παράδειγμα, αν ένα trial έχει τρεις διαφορετικές τιμές `gene_name` ('BRCA1', 'TP53', 'EGFR'), η στήλη `gene_name` στη compressed μορφή θα περιέχει τη λίστα ['BRCA1', 'TP53', 'EGFR'].

Αυτή η τεχνική διατηρεί **όλη την πληροφορία** που υπήρχε στο αρχικό dataset, απλώς την αναδιοργανώνει σε πιο συμπαγή μορφή. Το αποτέλεσμα είναι ένα DataFrame όπου ο αριθμός των γραμμών ισούται ακριβώς με τον αριθμό των μοναδικών trials, και αυτό επιβεβαιώνεται με τον έλεγχο του shape του `compressed_df`.

## Φιλτράρισμα Μεταβλητών Συνδυαστικής Δράσης

Μετά τη συμπίεση, πραγματοποιείται ένα κρίσιμο βήμα φιλτραρίσματος. Αφαιρούνται συγκεκριμένες στήλες που αφορούν **μετρήσεις συνδυαστικής δράσης φαρμάκων** (drug combination metrics), και δεν είναι κατάλληλες για το μοντέλο που προορίζεται να κατασκευαστεί. Οι στήλες αυτές περιλαμβάνουν:

- **Bliss matrices και παράγωγα**: `t.bliss_matrix`, `t.bliss_matrix_so`, `t.bliss_synergistic_wells`, `t.bliss_window_so` - μετρήσεις του Bliss independence model για συνέργεια
- **Delta μετρήσεις**: `t.delta_combo_maxe_day1`, `t.delta_maxe_lib1`, `t.delta_maxe_lib2` - διαφορές μεταξύ συνδυαστικών και μονο-θεραπευτικών αποτελεσμάτων
- **RMSE metrics**: `t.drug1_rmse`, `t.drug2_rmse` - σφάλματα προσαρμογής μοντέλων δόσης-απόκρισης
- **Concentration limits**: `drug.maxc`, `drug.minc` - όρια συγκέντρωσης φαρμάκων

Η αφαίρεση αυτών των μεταβλητών είναι ηθελημένη και βασίζεται στη λογική ότι το μοντέλο πρέπει να εκπαιδευτεί με δεδομένα που προέρχονται από **μονοθεραπεία** (single-agent treatment), ώστε να μπορεί να προβλέπει τη βασική βιολογική απόκριση χωρίς τη σύγχυση από αλληλεπιδράσεις φαρμάκων. Αυτό είναι σημαντικό για τη γενίκευση του μοντέλου και την ικανότητά του να προβλέπει αποτελέσματα σε νέους συνδυασμούς.

## Αφαίρεση Διπλοτύπων από Λίστες

Κατά τη διαδικασία aggregation, είναι πιθανό να δημιουργηθούν **διπλότυπες τιμές** μέσα στις λίστες. Αυτό μπορεί να συμβεί όταν το ίδιο trial συνδέεται πολλές φορές με την ίδια τιμή λόγω της δομής των queries στη βάση δεδομένων (π.χ. ένα γονίδιο που ανήκει σε πολλά pathways μπορεί να επαναλαμβάνεται).

Για να αντιμετωπιστεί αυτό, δημιουργείται η βοηθητική συνάρτηση **`unique_list()`**, η οποία:
1. Ελέγχει αν η είσοδος είναι λίστα
2. Αν ναι, μετατρέπει τη λίστα σε set (που εξ ορισμού δεν έχει διπλότυπα) και την επιστρέφει πίσω σε λίστα
3. Αν η τιμή δεν είναι λίστα (edge case), τη μετατρέπει σε single-element λίστα
4. Χειρίζεται με ασφάλεια τα null values

Η συνάρτηση αυτή εφαρμόζεται σε **όλες τις στήλες εκτός από τρεις**: `trial_id` (που είναι το group key), `gene_name`, και `tpm_value`. Αυτές οι δύο τελευταίες εξαιρούνται επειδή θέλουμε να **διατηρήσουμε τα duplicates** - για παράδειγμα, αν ένα γονίδιο εμφανίζεται πολλές φορές, κάθε εμφάνιση μπορεί να έχει διαφορετική TPM τιμή, και αυτή η πληροφορία είναι πολύτιμη.

## Unwrapping Single-Element Lists

Μετά την αφαίρεση των διπλοτύπων, πολλές λίστες μπορεί να καταλήξουν να έχουν **μόνο ένα στοιχείο**. Για παράδειγμα, αν ένα trial αφορά μόνο έναν τύπο καρκίνου (`cancer_type`), η λίστα θα είναι κάτι σαν `['Breast Cancer']`. Η αποθήκευση μιας μόνο τιμής ως λίστα είναι περιττή και μπορεί να δημιουργήσει προβλήματα στην επόμενη φάση επεξεργασίας.

Για να αντιμετωπιστεί αυτό, δημιουργείται η συνάρτηση **`unwrap_single_item_lists()`**, η οποία:
- Ελέγχει αν μια τιμή είναι λίστα με μήκος ακριβώς 1
- Αν ναι, επιστρέφει το μοναδικό στοιχείο της λίστας (π.χ. `['Breast Cancer']` → `'Breast Cancer'`)
- Αλλιώς, αφήνει την τιμή ανέπαφη (είτε είναι πολυ-στοιχειακή λίστα είτε άλλου τύπου δεδομένο)

Αυτή η μετατροπή κάνει το dataset πιο **καθαρό και ευανάγνωστο**, διακρίνοντας μεταξύ:
- **Scalar τιμών**: Μεταβλητές που έχουν μία μόνο τιμή ανά trial (π.χ. `cellline_name`, `cancer_type`)
- **List τιμών**: Μεταβλητές που πραγματικά έχουν πολλαπλές τιμές ανά trial (π.χ. `gene_name`, `tpm_value`, `pathway_name`)

Αυτός ο διαχωρισμός διευκολύνει σημαντικά το επόμενο στάδιο του data cleaning, όπου οι scalar μεταβλητές μπορούν να υποστούν κωδικοποίηση (encoding) ή κανονικοποίηση (scaling) απευθείας, ενώ οι λίστες χρειάζονται περαιτέρω επεξεργασία (π.χ. aggregation σε median, όπως γίνεται με το `tpm_value` στο data cleaning notebook).

## Εξαγωγή και Συνέχεια Pipeline

Το τελικό DataFrame αποθηκεύεται στο αρχείο **`model_dataset_compressed_v3.csv`**, το οποίο αποτελεί την έξοδο αυτού του σταδίου του pipeline. Αυτό το αρχείο έχει τη μορφή:
- Μία γραμμή ανά μοναδικό trial_id
- Scalar τιμές για μεταβλητές που έχουν μοναδική τιμή ανά trial
- Λίστες για μεταβλητές που έχουν πολλαπλές τιμές ανά trial
- Όλα τα duplicates αφαιρεμένα (εκτός από gene_name και tpm_value)
- Φιλτραρισμένες μεταβλητές που αφορούν συνδυαστική δράση

Το αρχείο αυτό αποτελεί τη βάση για το επόμενο στάδιο (το **data cleaning notebook**), όπου:
- Οι λίστες του `tpm_value` μετατρέπονται σε median τιμές
- Τα missing values συμπληρώνονται
- Οι αριθμητικές μεταβλητές υποβάλλονται σε scaling
- Οι κατηγορικές μεταβλητές κωδικοποιούνται (encoding)

Η όλη διαδικασία συμπίεσης είναι κρίσιμη γιατί επιτρέπει τη μετάβαση από μια **σχεσιακή αναπαράσταση δεδομένων** (όπως εξάγονται από τη graph database) σε μια **tabular αναπαράσταση** κατάλληλη για traditional machine learning αλγόριθμους. Χωρίς αυτό το στάδιο, το dataset θα ήταν πολύ αραιό (sparse) και δύσκολο να επεξεργαστεί.

In [1]:
import pandas as pd
from py2neo import Graph
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Φορτώνω τα raw data που πηρα απο το query
file_path= 'model_dataset.csv' #put your own downloaded file 
df= pd.read_csv(file_path)
df.head(2)

,trial_id,t.bliss_matrix,t.bliss_matrix_so,t.bliss_synergistic_wells,t.bliss_window,t.bliss_window_so,t.combo_maxe,t.day1_inhibition_scale,t.day1_intensity_mean,t.day1_viability_mean,...,cellline_name,cancer_type,c.tissue,gene_name,gene_id,tpm_value,pathway_name,drug_targeting_pathway,drug.maxc,drug.minc
0,47199,-0.063252,0.058931,13,0.070969,0.095355,0.639282,0.555151,5939.623377,0.444849,...,42-MG-BA,Glioblastoma,Central Nervous System,ATR,1.0,3.2494,Cell cycle,Palbociclib,10.0,0.01
1,47199,-0.063252,0.058931,13,0.070969,0.095355,0.639282,0.555151,5939.623377,0.444849,...,42-MG-BA,Glioblastoma,Central Nervous System,WEE1,2.0,4.7317,Cell cycle,Palbociclib,10.0,0.01


In [ ]:
#Βλέπω τι μέγεθος έχει το dataframe μου και ποσα διαφορετικά Trials έχω μέσα σε αυτές τις γραμμές
unique_trials = df['trial_id'].nunique()
print('Μοναδικά trial ids:', unique_trials)
print(df.shape)

Μοναδικά trial ids: 853
(560733, 32)


In [ ]:
#Ελέγχω για null τιμές
df.isnull().values.any()

True

In [24]:
print(df.isnull().sum())

trial_id                         0
t.bliss_matrix                   0
t.bliss_matrix_so                0
t.bliss_synergistic_wells        0
t.bliss_window                   0
t.bliss_window_so              663
t.combo_maxe                     0
t.day1_inhibition_scale      24306
t.day1_intensity_mean        24306
t.day1_viability_mean        24306
t.delta_combo_maxe_day1      24306
t.delta_maxe_lib1                0
t.delta_maxe_lib2                0
t.doubling_time              24306
t.drug1_ic50_ln                  0
t.drug1_maxe                     0
t.drug1_rmse                     0
t.drug2_ic50_ln                  0
t.drug2_maxe                     0
t.drug2_rmse                     0
t.growth_rate                24306
cellline_id                      0
cellline_name                    0
cancer_type                      0
c.tissue                         0
gene_name                        0
gene_id                       1706
tpm_value                        0
pathway_name        

In [ ]:
#Κάνω compress το df μου με βάση το trial_id
#όλες οι υπόλοιπες στήλες θα γίνουν λίστες με βάση αυτό

cols_to_agg= [col for col in df.columns if col != 'trial_id']
agg_dict={col:list for col in cols_to_agg}
compressed_df = df.groupby('trial_id', as_index=False).agg(agg_dict)
compressed_df.head(2)

,trial_id,t.bliss_matrix,t.bliss_matrix_so,t.bliss_synergistic_wells,t.bliss_window,t.bliss_window_so,t.combo_maxe,t.day1_inhibition_scale,t.day1_intensity_mean,t.day1_viability_mean,...,cellline_name,cancer_type,c.tissue,gene_name,gene_id,tpm_value,pathway_name,drug_targeting_pathway,drug.maxc,drug.minc
0,47199,"[-0.063251952, -0.063251952, -0.063251952, -0....","[0.05893089, 0.05893089, 0.05893089, 0.0589308...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[0.07096907, 0.07096907, 0.07096907, 0.0709690...","[0.09535543, 0.09535543, 0.09535543, 0.0953554...","[0.63928213, 0.63928213, 0.63928213, 0.6392821...","[0.555150906, 0.555150906, 0.555150906, 0.5551...","[5939.623377, 5939.623377, 5939.623377, 5939.6...","[0.444849094, 0.444849094, 0.444849094, 0.4448...",...,"[42-MG-BA, 42-MG-BA, 42-MG-BA, 42-MG-BA, 42-MG...","[Glioblastoma, Glioblastoma, Glioblastoma, Gli...","[Central Nervous System, Central Nervous Syste...","[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[3.2494, 4.7317, 6.0498, 5.0904, 2.5484, 2.403...","[Cell cycle, Cell cycle, Cell cycle, Cell cycl...","[Palbociclib, Palbociclib, Palbociclib, Palboc...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0..."
1,47226,"[-0.088889257, -0.088889257, -0.088889257, -0....","[0.077368031, 0.077368031, 0.077368031, 0.0773...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[0.031542684, 0.031542684, 0.031542684, 0.0315...","[0.115645554, 0.115645554, 0.115645554, 0.1156...","[0.460322608, 0.460322608, 0.460322608, 0.4603...","[0.059249023, 0.059249023, 0.059249023, 0.0592...","[19426.83442, 19426.83442, 19426.83442, 19426....","[0.940750977, 0.940750977, 0.940750977, 0.9407...",...,"[ALL-SIL, ALL-SIL, ALL-SIL, ALL-SIL, ALL-SIL, ...","[T-Lymphoblastic Leukemia, T-Lymphoblastic Leu...","[Haematopoietic and Lymphoid, Haematopoietic a...","[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[4.3125, 6.0199, 7.8476, 4.9887, 3.8135, 1.922...","[Cell cycle, Cell cycle, Cell cycle, Cell cycl...","[Palbociclib, Palbociclib, Palbociclib, Palboc...","[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....","[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0..."


In [ ]:
#Ελέγχω το μέγεθος του καινούριου df ώστε να είμαι σίγουρη ότι οι σειρές 
#αντιστοιχούν στα μοναδικά trial_id
print(compressed_df.shape)

(853, 32)


In [ ]:
#Έπειτα από επανεξέταση του documentation καταλήγω να κάνω drop ττις εξής στήλες για να μην επηρεάσουν το μοντέλο μου
#οι περισσότερες από τις στήλες που έγιναν drop είναι γιατί τα values τους προέρχονται από το συνδιασμο
#φαρμακων και οχι απο μονοθεραπεια

new_df = compressed_df.drop(columns=['t.bliss_matrix','t.bliss_matrix_so','t.bliss_synergistic_wells',	't.bliss_window_so', 
        't.delta_combo_maxe_day1', 't.delta_maxe_lib1',
       't.delta_maxe_lib2', 't.drug1_rmse', 't.drug2_rmse', 'drug.maxc', 'drug.minc'])

In [29]:
new_df.shape

(853, 21)

In [28]:
new_df.columns

Index(['trial_id', 't.bliss_window', 't.combo_maxe', 't.day1_inhibition_scale',
       't.day1_intensity_mean', 't.day1_viability_mean', 't.doubling_time',
       't.drug1_ic50_ln', 't.drug1_maxe', 't.drug2_ic50_ln', 't.drug2_maxe',
       't.growth_rate', 'cellline_id', 'cellline_name', 'cancer_type',
       'c.tissue', 'gene_name', 'gene_id', 'tpm_value', 'pathway_name',
       'drug_targeting_pathway'],
      dtype='object')

In [ ]:
#Συναρτηση που αφαιρεί διπλότυπα από λίστες και μετατρέπει τα non-list values σε λίστες

def unique_list(x):
    if isinstance(x, list):
        return list(set(x))
    else:
        return [x] if pd.notnull(x) else []

In [ ]:
#Εφαρμόζω τη συναρτηση σε ολα τα δεδομένα που συμπίεσα, εκτος απο 3 στηλες για να μη χαθει πληροφορια

exclude_cols = {'trial_id', 'gene_name', 'tpm_value'}

cols_to_process =[col for col in new_df.columns if col not in exclude_cols]

for col in cols_to_process:
    new_df[col] = new_df[col].apply(unique_list)

new_df.head(2)

,trial_id,t.bliss_window,t.combo_maxe,t.day1_inhibition_scale,t.day1_intensity_mean,t.day1_viability_mean,t.doubling_time,t.drug1_ic50_ln,t.drug1_maxe,t.drug2_ic50_ln,...,t.growth_rate,cellline_id,cellline_name,cancer_type,c.tissue,gene_name,gene_id,tpm_value,pathway_name,drug_targeting_pathway
0,47199,[0.07096907],[0.63928213],[0.555150906],[5939.623377],[0.444849094],[63.85180142],[1.239688417],[0.6130709],[3.863034059],...,[1.127611099],[1465],[42-MG-BA],[Glioblastoma],[Central Nervous System],"[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[3.2494, 4.7317, 6.0498, 5.0904, 2.5484, 2.403...","[PI3K/MTOR signaling, Cell cycle]","[Palbociclib, Capivasertib]"
1,47226,[0.031542684],[0.460322608],[0.059249023],[19426.83442],[0.940750977],[839.3959358],[2.562844612],[0.44856901],[2.320665612],...,[0.085775969],[2263],[ALL-SIL],[T-Lymphoblastic Leukemia],[Haematopoietic and Lymphoid],"[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[4.3125, 6.0199, 7.8476, 4.9887, 3.8135, 1.922...","[PI3K/MTOR signaling, Cell cycle]","[Palbociclib, Capivasertib]"


In [ ]:
#Συνάρτηση που μετατρέπει όσες στήλες είναι λιστες σε απλα values, και μετά εφαρμόζεται

def unwrap_single_item_lists(x):
    if isinstance(x, list) and len(x) == 1:
        return x[0]
    else:
        return x
    
for col in cols_to_process:
    new_df[col] = new_df[col].apply(unwrap_single_item_lists)

new_df.head(2)

,trial_id,t.bliss_window,t.combo_maxe,t.day1_inhibition_scale,t.day1_intensity_mean,t.day1_viability_mean,t.doubling_time,t.drug1_ic50_ln,t.drug1_maxe,t.drug2_ic50_ln,...,t.growth_rate,cellline_id,cellline_name,cancer_type,c.tissue,gene_name,gene_id,tpm_value,pathway_name,drug_targeting_pathway
0,47199,0.070969,0.639282,0.555151,5939.623377,0.444849,63.851801,1.239688,0.613071,3.863034,...,1.127611,1465,42-MG-BA,Glioblastoma,Central Nervous System,"[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[3.2494, 4.7317, 6.0498, 5.0904, 2.5484, 2.403...","[PI3K/MTOR signaling, Cell cycle]","[Palbociclib, Capivasertib]"
1,47226,0.031543,0.460323,0.059249,19426.83442,0.940751,839.395936,2.562845,0.448569,2.320666,...,0.085776,2263,ALL-SIL,T-Lymphoblastic Leukemia,Haematopoietic and Lymphoid,"[ATR, WEE1, AURKB, ATM, BRCA2, ABRAXAS1, ATRIP...","[1.0, 2.0, 3.0, 4.0, 6.0, 10.0, 13.0, 15.0, 19...","[4.3125, 6.0199, 7.8476, 4.9887, 3.8135, 1.922...","[PI3K/MTOR signaling, Cell cycle]","[Palbociclib, Capivasertib]"


In [ ]:
#Αποθηκέυω το αρχείο locally

new_df.to_csv('model_dataset_compressed_v3.csv', index=False)